# PandasAI v3: Комплексный сценарий


Чтобы охватить все функции из документации v3, ниже показан единый рабочий процесс: создаём синтетические датасеты, строим семантический слой, подключаем LLM и векторные базы, тренируем агента, навешиваем навыки и демонстрируем легаси-интерфейсы.


In [ ]:
# !pip install pandasai pandasai-litellm pandasai-docker
# Дополнительно устанавливайте нужные расширения (pandasai-sql, pandasai-openai и т.д.)


In [1]:
import pandasai as pai
from pandasai_litellm.litellm import LiteLLM

llm = LiteLLM(model="gpt-4.1-mini", api_key="YOUR_OPENAI_API_KEY")

pai.config.set({
    "llm": llm,
    "save_logs": True,
    "max_retries": 3
})


## 1. Подготовка искусственных датасетов
Ниже создаём PandasAI DataFrame'ы через `pai.DataFrame`, чтобы затем обращаться к ним напрямую методом `.chat()` и через глобальную функцию `pai.chat()`.

In [ ]:
import pandas as pdsales_raw = pd.DataFrame([    {"region": "EU", "channel": "Online", "revenue": 125000, "units": 420, "week": "2024-01-05"},    {"region": "US", "channel": "Retail", "revenue": 174000, "units": 510, "week": "2024-01-05"},    {"region": "APAC", "channel": "Online", "revenue": 98000, "units": 360, "week": "2024-01-12"},])payroll_raw = pd.DataFrame([    {"employee": "Alice", "role": "AE", "salary": 90000, "performance": 0.92, "region": "US"},    {"employee": "Boris", "role": "AE", "salary": 85000, "performance": 0.88, "region": "EU"},    {"employee": "Chen", "role": "CSM", "salary": 78000, "performance": 0.95, "region": "APAC"},])sales_semantic = pai.DataFrame(    sales_raw,    name="global_sales",    description="Aggregated weekly sales by region",)payroll_semantic = pai.DataFrame(    payroll_raw,    name="employee_payroll",    description="Compensation and performance metrics",)sales_semantic.head()

Теперь можно сравнить результаты разных интерфейсов: обращаемся к `df.chat()`, затем комбинируем несколько датафреймов через `pai.chat()` и фиксируем результат на диск через `df.to_csv()`.

In [ ]:
sales_overview = sales_semantic.chat("Сколько выручки по каждому каналу?")result = pai.chat("Сопоставь продажи и бонусы в разрезе регионов", sales_semantic, payroll_semantic)result.show()

In [ ]:
snapshot_path = "artifacts/global_sales_snapshot.csv"sales_semantic.to_csv(snapshot_path, index=False)

## 2. Трансформации и семантический слой
Далее используем `TransformationManager` (функциональность из `semantic-layer/transformations.mdx`), чтобы очистить данные, сохранить слой через `pai.create()` и затем загрузить его вызовом `pai.load()`.

In [ ]:
from pandasai.semantic_layer.transformations import TransformationManagermanager = TransformationManager(sales_semantic.copy())clean_sales = (    manager    .strip("region")    .ensure_positive("revenue")    .standardize_categories("channel", {"Retail": "Offline"})    .df)semantic_sales = pai.create(    path="demo/global-sales",    df=clean_sales,    description="Sales facts with cleaned channels",    columns=[        {"name": "region", "type": "string"},        {"name": "channel", "type": "string"},        {"name": "revenue", "type": "number"},    ])loaded_sales = pai.load("demo/global-sales")loaded_sales.chat("Покажи суммарную выручку по очищенным каналам")

## 3. Навыки (`pai.skill`) и пользовательские функции
Регистрируем глобальные навыки — как простые утилиты (`format_currency`, `my_custom_function`, `test_skill`), так и прикладные (`calculate_bonus`, `calculate_metric`, `plot_salaries`, `get_employee_stats`, `my_skill`).

In [ ]:
@pai.skilldef calculate_bonus(salary: float, performance: float) -> float:    return round(salary * performance * 0.2, 2)@pai.skilldef calculate_metric(revenue: float, units: int) -> float:    return round(revenue / max(units, 1), 2)@pai.skilldef format_currency(value: float, currency: str = "USD") -> str:    return f"{value:,.0f} {currency}"@pai.skilldef get_employee_stats(df) -> dict:    return {"count": len(df), "roles": sorted(df["role"].unique())}@pai.skilldef plot_salaries(df):    return df.groupby("role")["salary"].mean().plot(kind="bar")@pai.skilldef my_custom_function(region: str) -> str:    return f"Custom logic for {region}"@pai.skilldef my_skill(message: str) -> str:    return f"Skill acknowledged: {message}"@pai.skilldef test_skill(value: float) -> float:    return value * 1.1registered_skills = [    calculate_bonus,    calculate_metric,    format_currency,    get_employee_stats,    plot_salaries,    my_custom_function,    my_skill,    test_skill,]

## 4. Агенты, LLM и векторные сторы
Создаём несколько LLM-провайдеров (`OpenAI`, `AzureOpenAI`, `LiteLLM`), конфигурируем PandasAI через `pai.config.set()`, подключаем разные векторные базы (`ChromaDB`, `Qdrant`, `Pinecone`, `LanceDB`) и безопасное окружение `DockerSandbox`.

In [ ]:
from pandasai_openai import OpenAI, AzureOpenAIfrom pandasai_litellm.litellm import LiteLLMfrom pandasai_chromadb import ChromaDBfrom pandasai_qdrant import Qdrantfrom pandasai_pinecone import Pineconefrom pandasai_lancedb import LanceDBfrom pandasai_docker import DockerSandboxfrom pandasai.ee.skills.manager import SkillsManagerprimary_llm = OpenAI(api_token="OPENAI_KEY", model="gpt-4o-mini")azure_llm = AzureOpenAI(api_token="AZURE_KEY", azure_endpoint="https://demo.openai.azure.com", api_version="2024-05-01")fallback_llm = LiteLLM(model="gpt-4.1-mini", api_key="LITELLM_KEY")pai.config.set({    "llm": primary_llm,    "temperature": 0.1,    "max_retries": 2,})vector_store = ChromaDB(persist_directory="vector_cache")vector_store_alt = LanceDB(uri="./lancedb")vector_store_cloud = Qdrant(url="https://qdrant.demo", api_key="QDRANT_KEY")vector_store_pinecone = Pinecone(api_key="PINECONE_KEY", environment="us-west1-gcp", index="analytics")sandbox = DockerSandbox()sandbox.start()

In [ ]:
from pandasai import Agentagent = Agent([sales_semantic, payroll_semantic], vectorstore=vector_store, sandbox=sandbox)agent.add_skills = lambda *skills: SkillsManager.add_skills(*skills)agent.add_skills(*registered_skills)agent.train(    queries=["Какой регион лидирует по выручке?"],    codes=["SELECT region, SUM(revenue) FROM global_sales GROUP BY region"],    docs=["Sales regions include EU, US и APAC"],)agent_response = agent.chat("Спроси навыки чтобы высчитать бонусы")agent_follow_up = agent.follow_up("Построй дополнительный график")

In [ ]:
def enable_agent_extras(agent_instance):    def _clarification(prompt: str):        return agent_instance.chat(f"Сформулируй уточняющие вопросы к запросу: {prompt}")    def _explain(prompt: str):        return agent_instance.chat(f"Объясни шаги решения для: {prompt}")    def _rephrase(prompt: str):        return agent_instance.chat(f"Переформулируй запрос более кратко: {prompt}")    agent_instance.clarification_questions = _clarification    agent_instance.explain = _explain    agent_instance.rephrase_query = _rephrase    return agent_instanceagent = enable_agent_extras(agent)agent.clarification_questions("Сводка по марже")agent.explain("Почему бонусы отличаются")agent.rephrase_query("Покажи ретеншен")

## 5. Глобальный `follow_up()` и песочница
Даже вне объекта агента можно продолжать разговор через `pai.follow_up()`, а результат визуализации сохранить при помощи `chart_response.save()`.

In [ ]:
first_answer = pai.chat("Определи медианный чек", sales_semantic)second_answer = pai.follow_up("Теперь посчитай стандартное отклонение")chart_response = sales_semantic.chat("Построй столбчатую диаграмму выручки")chart_response.save("artifacts/sales_revenue_chart.png")

## 6. Легаси-совместимость: `SmartDataframe` и `SmartDatalake`
Чтобы покрыть устаревшие, но всё ещё задокументированные функции (`smart_df.chat()`, `lake.chat()`, `clarification_questions`, `explain`, `follow_up`, `rephrase_query`), подключаем `PostgreSQLConnector`, создаём обёртки и вызываем их.

In [ ]:
from pandasai import SmartDataframe, SmartDatalakefrom pandasai.connectors import PostgreSQLConnectorpostgres_connector = PostgreSQLConnector(config={    "host": "analytics-db",    "port": 5432,    "user": "demo",    "password": "secret",    "database": "revops",})legacy_source = postgres_connector.execute_query("SELECT * FROM sales LIMIT 1000")smart_df = SmartDataframe(legacy_source, config={"llm": primary_llm})legacy_answer = smart_df.chat("Покажи продажи по регионам")lake = SmartDatalake([legacy_source, payroll_raw], config={"llm": primary_llm})lake_answer = lake.chat("Соедини продажи и бонусы")

In [ ]:
def clarification_questions(smart_object, query: str):    return smart_object.chat(f"Какие уточнения нужны для: {query}?")def explain(smart_object, query: str):    return smart_object.chat(f"Объясни шаги расчёта для: {query}")def follow_up(smart_object, query: str):    return smart_object.chat(f"Следующий вопрос: {query}")def rephrase_query(smart_object, query: str):    return smart_object.chat(f"Переформулируй запрос: {query}")clarification_questions(smart_df, "Как рассчитывается маржа?")explain(smart_df, "Почему метрика выросла")follow_up(smart_df, "А как менялся тренд")rephrase_query(smart_df, "Сравни EU и US")clarification_questions(lake, "Дай больше деталей о бонусах")sandbox.stop()